In [ ]:
pip install pandas
pip install qiskit==0.39.2
pip install 

In [2]:
from pVQD_cluster import pVQD
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info.states import statevector
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import L_BFGS_B
from ansatze import hweff_ansatz, hweff_param_ansatz
from qiskit import Aer
from scipy.sparse import diags
import shin_metiu_full as sf
import pennylane.numpy as np
import scipy.linalg as sl
import argparse
import json
from auto_plotter import autoplotter
import pennylane as qml
import copy

resolution = 16
L =19
padding = 5

ModuleNotFoundError: No module named 'pandas'

In [157]:
def hweff_ansatz(p, n_spins=4, depth=4):
    count=0
    for j in range(depth):

        if j % 2 == 0:
            # Rx - Rzz block
            for i in range(n_spins):
                qml.RX(p[count],wires =  i)
                count = count + 1


            for i in range(n_spins - 1):
                qml.IsingZZ(p[count], wires = [i, i + 1])
                count = count + 1


        if j % 2 == 1:
            for i in range(n_spins):
                qml.RY(p[count], wires =  i)
                count = count + 1


            for i in range(n_spins - 1):
                qml.IsingZZ(p[count], wires= [i, i + 1])
                count = count + 1


    # Final block to close the ansatz
    if depth % 2 == 1:
        for i in range(n_spins):
            qml.RY(p[count], wires = i)
            count = count + 1
    if depth % 2 == 0:
        for i in range(n_spins):
            qml.RX(p[count], wires = i)
            count = count + 1
# hweff_ansatz([1]*1000,4,4)

In [158]:
def prepare_state(r_f, r_l, r_r, resolution=resolution, L=L, padding=padding):
    full = sf.ShinMetiu(
        nstates=resolution,
        L=L,
        Rf=r_f,
        Rl=r_l,
        Rr=r_r,
        mass=1836.0,
        m_el=1.0,
        nel=resolution,
        padding=padding,
    )
    steps = resolution
    dx = L / (steps + 1)
    L2 = L / 2 - dx
    xgrid = np.linspace(-L2, L2, steps)
    return full, xgrid

def prep_ham(r_nuc, r_f, r_l, r_r, resolution=resolution, L=L, padding=padding):
    full, xgrid = prepare_state(r_f, r_l, r_r, resolution, L=L, padding=padding)
    H_diag, H_off_diag = full.H(np.array([r_nuc]))
    V_nuc = full.V_nuc(np.array([r_nuc]))
    dHdr = full.dV_el(np.array([r_nuc]))
    dHdr_nuc = full.dV_nuc(np.array([r_nuc]))
    En, Q = sl.eigh_tridiagonal(H_diag, H_off_diag, eigvals_only=False)
    H = diags([H_diag, H_off_diag, H_off_diag], [0, 1, -1]).toarray()
    return H, En, Q, dHdr, xgrid, dHdr_nuc, V_nuc

H, En, Q, dHdr, xgrid, dHdr_nuc, V_nuc = prep_ham(-1.9,5,3.1,4)
H = qml.pauli_decompose(H)
dHdr = qml.pauli_decompose(dHdr)

In [159]:
max_iterations = 500
conv_tol = 1e-06
step_size = 0.01

In [66]:
dev = qml.device("lightning.qubit", wires=4)
opt = qml.QNGOptimizer(stepsize=step_size)

@qml.qnode(dev)
def cost_fn(params):
    hweff_ansatz(params, n_spins=4, depth=4)
    return qml.expval(H)

@qml.qnode(dev)
def force(params):
    hweff_ansatz(params, n_spins=4, depth=4)
    return qml.expval(dHdr)


params = best_params

gd_param_history = [params]
gd_cost_history = []

for n in range(max_iterations):

    # Take step
    params, prev_energy = opt.step_and_cost(cost_fn, params)
    gd_param_history.append(params)
    gd_cost_history.append(prev_energy)

    energy = cost_fn(params)

    # Calculate difference between new and old energies
    conv = np.abs(energy - prev_energy)

    if n % 20 == 0:
        print(
            "Iteration = {:},  Energy = {:.8f} Ha,  Convergence parameter = {"
            ":.8f} Ha".format(n, energy, conv)
        )

    if conv <= conv_tol:
        break

print()
print("Final value of the energy = {:.8f} Ha".format(energy))
print("Number of iterations = ", n)

Iteration = 0,  Energy = -0.45075158 Ha,  Convergence parameter = 0.00005880 Ha
Iteration = 20,  Energy = -0.45188842 Ha,  Convergence parameter = 0.00005514 Ha
Iteration = 40,  Energy = -0.45295613 Ha,  Convergence parameter = 0.00005186 Ha
Iteration = 60,  Energy = -0.45396170 Ha,  Convergence parameter = 0.00004890 Ha
Iteration = 80,  Energy = -0.45491126 Ha,  Convergence parameter = 0.00004623 Ha
Iteration = 100,  Energy = -0.45581016 Ha,  Convergence parameter = 0.00004382 Ha
Iteration = 120,  Energy = -0.45666319 Ha,  Convergence parameter = 0.00004163 Ha
Iteration = 140,  Energy = -0.45747456 Ha,  Convergence parameter = 0.00003964 Ha
Iteration = 160,  Energy = -0.45824805 Ha,  Convergence parameter = 0.00003783 Ha
Iteration = 180,  Energy = -0.45898697 Ha,  Convergence parameter = 0.00003617 Ha
Iteration = 200,  Energy = -0.45969423 Ha,  Convergence parameter = 0.00003465 Ha
Iteration = 220,  Energy = -0.46037243 Ha,  Convergence parameter = 0.00003326 Ha
Iteration = 240,  Ener

In [ ]:
best_params= [-1.1667490045980744,
 0.7604082770626044,
 -0.5611703567392711,
 -0.2243868640850896,
 -1.4773023685037536,
 -1.4077197800956138,
 1.7376202712412656,
 0.06430769186502076,
 -1.913574339237776,
 0.9761327877750625,
 1.423704771076856,
 -0.720433790279799,
 0.23409740329218237,
 -0.22846618783507636,
 0.34009767312447514,
 1.5136908411406615,
 0.18732559967879697,
 -0.3660627420955632,
 0.08556223920475048,
 -0.21141052587099254,
 0.09130290454581434,
 -0.6464953162611212,
 -0.8009078545305401,
 0.5622728039545047,
 0.36448868187384126,
 -1.5238665884705254,
 0.5792536316391361,
 0.6029133789744988,
 1.1030321344741656,
 1.0573339420949215,
 -1.7515692359716422,
 -0.06593971455031955]

In [161]:
@qml.qnode(dev)
def force(params):
    hweff_ansatz(params)
    return qml.expval(dHdr)

F_el = -force(best_params)/1836

In [162]:
comp = np.zeros(2**4,dtype=complex,requires_grad=True)
comp[0]=1


In [ ]:
log_data = {}

In [ ]:
log_data['ideal_coefficients'] = ideal.store_c.tolist()
log_data["ideal_forces_el"] = ideal.store_fel.tolist()
log_data["ideal_forces_nuc"] = ideal.store_fnuc.tolist()
log_data["ideal_tot_forces"] = ideal.store_f.tolist()
log_data["ideal_velocities"] = ideal.store_v.tolist()
log_data["ideal_positions"] = ideal.store_xs.tolist()
log_data["ideal_energy_el"] = ideal.store_energy_el.tolist()
log_data["ideal_energy_Tnuc"] = ideal.store_energy_Tnuc.tolist()
log_data["ideal_energy_Vnuc"] = ideal.store_energy_Vnuc.tolist()
log_data["ideal_energy"] = (ideal.store_energy_el+ideal.store_energy_Tnuc+ideal.store_energy_Vnuc).tolist()
### Exact results
log_data['exact_coefficients'] = exact.store_c.tolist()
log_data["exact_forces_el"] = exact.store_fel.tolist()
log_data["exact_forces_nuc"] = exact.store_fnuc.tolist()
log_data["exact_tot_forces"] = exact.store_f.tolist()
log_data["exact_velocities"] = exact.store_v.tolist()
log_data["exact_positions"] = exact.store_xs.tolist()
log_data["exact_energy_el"] = exact.store_energy_el.tolist()
log_data["exact_energy_Tnuc"] = exact.store_energy_Tnuc.tolist()
log_data["exact_energy_Vnuc"] = exact.store_energy_Vnuc.tolist()
log_data["exact_energy"] = (exact.store_energy_el+exact.store_energy_Tnuc+exact.store_energy_Vnuc).tolist()


In [ ]:
log_data["QC_energy_el"] = []
log_data["dt"] = initial_parameters["dt"]
log_data["depth"] = self.depth
log_data["r_r"] = initial_parameters["r_r"]
log_data["r_f"] = initial_parameters["r_f"]
log_data["r_l"] = initial_parameters["r_l"]
log_data["L"] = initial_parameters["L"]
log_data["res"] = initial_parameters["res"]
log_data["trotter_steps"] = trotter_steps
log_data["parameterized"] = initial_parameters["parameterized"]
log_data["padding"] = initial_parameters["padding"]
log_data["fidelity_to_ideal"] = [self.overlap[0]]
log_data["fidelity_to_exact"] = [1]
log_data["QC_energy_Tnuc"] = []
log_data["QC_energy_Vnuc"] = []
log_data["QC_coefficients"] = []
log_data["QC_energy"] = [] 
log_data['ideal_energy'] = []
log_data['total_energy'] = []

log_data["init_F"] = initial_fidelities
log_data["QC_positions"] = positions
log_data["QC_velocities"] = velocities
log_data["QC_tot_forces"] = forces
log_data["QC_forces_nuc"] = forces_nuc
log_data["QC_forces_el"] = forces_el
log_data["final_F"] = fidelities
log_data["err_init_F"] = err_init_fid
log_data["err_fin_F"] = err_fin_fid
log_data["iter_number"] = counter
log_data["times"] = times
log_data["params"] = list(params)
log_data["tot_steps"] = [tot_steps]
log_data["final_i"] = i


In [234]:
dev = qml.device("default.qubit", wires=4)
@qml.qnode(dev)
def pvqd_state(init_params, ham=H, comp=comp):
    const = copy.copy(init_params)
    hweff_ansatz(const, n_spins=4, depth=4)
    qml.ApproxTimeEvolution(ham,0.05,1)
    qml.adjoint(hweff_ansatz)(init_params, n_spins=4, depth=4)
    return qml.state()

def pvqd_cost(init_params, n_spins=4, depth=4):
    state = pvqd_state(init_params, ham=H, comp=comp)
    f = qml.math.fidelity(state,comp)
    return 1.0-f

@qml.qnode(dev)
def q_dHdr(params,dHdr):
    hweff_ansatz(params)
    return qml.expval(dHdr)

@qml.qnode(dev)
def energy(params, H):
    hweff_ansatz(params, n_spins=4, depth=4)
    return qml.expval(H)

def force_el(params, dHdr):
    return -q_dHdr(params, dHdr)/1836

In [235]:
x = [-1.9]
v = [0.02]
timesteps=1
params = best_params
params_hist = [best_params]
initial_fidelity = []
final_fidelity = []
opt =   qml.AdamOptimizer()

H_i, _, _, dHdr_i, _, dHdr_nuc, V_nuc = prep_ham(x,5,3.1,4)
H_i = qml.pauli_decompose(H_i)
dHdr_i = qml.pauli_decompose(dHdr_i)

comp = np.zeros(2**4,dtype=complex,requires_grad=True)
comp[0]=1


for x in range(timesteps):

    for n in range(161): # Optimize to 0 state
        params, how_0 = opt.step_and_cost(pvqd_cost, params)
        if n==0:
            initial_fidelity.append(1-how_0)
        if how_0 <= 1e-7:
            break
    final_fidelity.append(1-how_0)
    
    
    params_hist.append(params)
